In [3]:
import pandas as pd


In [18]:
neg = pd.read_csv('Data_0.csv').dropna()
neu = pd.read_csv('Data_1.csv')
pos = pd.read_csv('Data_2.csv').dropna()

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [49]:
neu = neu.iloc[:,:3]
neu = neu.dropna()

,Date,Title,Labeling
0,2020.07.07.,‘2NE1’ 공민지 “SM 오디션 먼저 봐…지용·태양 오빠가 잠실 놀이공원서 서...,1.0
1,7일,‘7번방 예승이’ 갈소원 폭풍성장 근황 “러블리함의 끝”,1.0
2,7일,"‘7번방 예승이’ 갈소원, 폭풍성장 근황 공개… 정변의 아이콘 등극",1.0
3,7일,"‘7번방의 선물’ 갈소원, 폭풍성장 근황 ‘입이 쩍’…네티즌 “정말 예쁘게...",1.0
4,2020.07.16.,"‘개성파 배우’ 박충환, 네오스엔터테인먼트와 전속계약 체결",1.0
...,...,...,...
3192,2020.07.30.,"YG를 향한 팬들의 ‘트럭시위’, 계속되는 이유는? [이쓔시개]",1.0
3193,2020.08.03.,"YG에 1000억 투자했던 네이버, 이번에는 SM…""팬클럽 서비스 및 콘텐츠 강화""",1.0
3194,2020.09.06.,"YG엔터테인먼트, 블랙핑크 정규앨범으로 해외 음반판매 새 기록 도전",1.0
3195,2020.07.02.,"YG엔터테인먼트, 장기용 입대설에 '묵묵부답'",1.0


In [51]:
print(len(pos))
print(len(neu))
print(len(neg))
# 나중에 SMOTE를 이용해서 neg 를 뻥튀기 해서 시험 해야함 . 

3268
3197
650


In [52]:
pos['Labeling'] = 2
neu['Labeling'] = 1
neg['Labeling'] = 0

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
pos_train, pos_test = train_test_split(pos, test_size=0.3, shuffle=True, random_state=1004)
neu_train, neu_test = train_test_split(neu, test_size=0.3, shuffle=True, random_state=1004)
neg_train, neg_test = train_test_split(neg, test_size=0.3, shuffle=True, random_state=1004)

In [57]:
print(len(pos_train))
print(len(pos_test))
print(len(neu_train))
print(len(neu_test))
print(len(neg_train))
print(len(neg_test))

2287
981
2237
960
455
195


In [54]:
# Smote를 사용해본다
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
pos_train_over,neg_train_over = smote.fit_sample(pos_train,neg_train)

TypeError: '<' not supported between instances of 'int' and 'str'

In [59]:
train = pd.concat([pos_train, neu_train, neg_train])
test = pd.concat([pos_test, neu_test,neg_test])
print(len(train))
print(len(test))

4979
2136


In [41]:
train_df = train.sample(frac=0.4, random_state=999)
test_df = test.sample(frac=0.4, random_state=999)

In [48]:
train_df.columns = ['id','document','label']
test_df.columns = ['id','document','label']
test_df

,id,document,label
2454,2020.09.12.,"컴백' YG 트레저 하루토X소정환, 팬 사랑 부르는 '산뜻' 비주얼",1
573,2020.06.23.,"한서희, 3개월만에 ""다시"" 돌아온 SNS…'공익신고자'의 밝은 근황",0
174,2020.09.09.,[TEN 포토] 양현석 '원정도박 혐의로 법원 출석',0
2939,2020.07.24.,"YG 신예' 트레저, 데뷔곡 안무 스포…멤버 12인 각양각색 일상 공개",1
1472,2020.07.17.,"블랙핑크, ‘How You Like That’스페셜 에디션 오늘 발매",1
...,...,...,...
3216,2020.07.28.,"YG, 블랙핑크 댄스커버 대회 성료…필리핀·미국 비롯 100개국 2200팀 참여",1
982,2020.09.03.,"블랙핑크 10억뷰, '킬 디스 러브'로 두 번째 10억뷰 ""K팝 최초""",1
179,2020.08.31.,"""우리에게 눕방 없다""..트레저, 100만명·2억하트 돌파..초대형 아이돌의 탄생...",1
1245,2일,"블랙핑크, '3억 뷰' 돌파!...K팝 아티스트 중 유일",1


In [49]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [50]:
class NsmcDataset(Dataset):
    ''' Naver Sentiment Movie Corpus Dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 2]
        return text, label

In [51]:
nsmc_train_dataset = NsmcDataset(train_df)
train_loader = DataLoader(nsmc_train_dataset, batch_size=2, shuffle=True, num_workers=2)

In [52]:
device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device)

100%|██████████| 714314041/714314041 [01:46<00:00, 6697564.69B/s] 


KeyboardInterrupt: 

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0


model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()

        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

In [ ]:
# evaluation
model.eval()

nsmc_eval_dataset = NsmcDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)
 